In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from matplotlib.patches import Arc
from pylab import rcParams
rcParams['figure.figsize'] = 14/2.54, 10/2.54
matplotlib.font_manager.FontProperties(family='Helvetica',size=11);

# 9.1 Analysing happiness across countries

In this exercise, we will consider the data set `data/happy.csv` with data from the World Happiness Report. For details see: https://worldhappiness.report/ed/2019/changing-world-happiness/

## Dataset

`GDP per capita` is in terms of Purchasing Power Parity (PPP) adjusted to constant 2011 international dollars, taken from the World Development Indicators (WDI) released by the World Bank on November 14, 2018. The equation uses the natural log of GDP per capita, as this form fits the data significantly better than GDP per capita.

The time series of `healthy life expectancy` at birth are constructed based on data from the World Health Organization (WHO) Global Health Observatory data repository, with data available for 2005, 2010, 2015, and 2016. To match this report’s sample period, interpolation and extrapolation are used. 

`Social support` is the national average of the binary responses (either 0 or 1) to the Gallup World Poll (GWP) question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”

`Freedom to make life choices` is the national average of binary responses to the GWP question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”

`Generosity` is the residual of regressing the national average of GWP responses to the question “Have you donated money to a charity in the past month?” on GDP per capita.

`Perceptions of corruption` are the average of binary answers to two GWP questions: “Is corruption widespread throughout the government or not?” and “Is corruption widespread within businesses or not?” Where data for government corruption are missing, the perception of business corruption is used as the overall corruption-perception measure.

## a)
Load and familiarize yourself with the data set. We rename them because statsmodels needs variables to not include spaces.

In [ ]:
#Read in the data
#happy = pd.read_csv("data/happy.csv",delimiter=';') 
happy = pd.read_csv('https://uu-sml.github.io/course-sml-public/data/happy.csv', delimiter=';')
happy.head()

In [ ]:
happy.rename(columns = {'Social support':'SocialSupport'}, inplace = True) 
happy.rename(columns = {'Life Ladder': 'LifeLadder'}, inplace = True) 
happy.rename(columns = {'Perceptions of corruption':'Corruption'}, inplace = True) 
happy.rename(columns = {'Log GDP per capita': 'LogGDP'}, inplace = True) 
happy.rename(columns = {'Healthy life expectancy at birth': 'LifeExp'}, inplace = True) 
happy.rename(columns = {'Freedom to make life choices': 'Freedom'}, inplace = True) 

#In this exercise we will just analyse one year. 2017.
df=happy.loc[happy['Year'] == 2017]
df=df.dropna()
df.head()

## b)
The code below fits a linear regression model to predict life ladder (happiness) as a function of social support. Edit the code to fit a third order polynomial. What model would you suggest to use?

In [ ]:
#Fit model.
model_fit=smf.ols(formula='LifeLadder ~  SocialSupport - 1 ', data=df).fit()
print(model_fit.summary())        
b=model_fit.params

In [ ]:
#Compute predictions.
x=np.arange(0.4,1,step=0.01)
y=b[0]*x

In [ ]:
#Fit model polynomial


In [ ]:
#Compute predictions.


In [ ]:
#Plot social support and life ladder data.
fig,ax=plt.subplots(num=1)
ax.plot(  'SocialSupport','LifeLadder', data=df, linestyle='none', markersize=4, marker='o', color='grey')
countries=['United Kingdom','Croatia','Benin','Finland','Afghanistan']
for country in countries:
    ci=np.where(df['Country name']==country)[0][0]
    ax.plot(  df.iloc[ci]['SocialSupport'],df.iloc[ci]['LifeLadder'], linestyle='none', markersize=6, marker='o', color='black')
    if country=='United Kingdom':
        ax.text(  df.iloc[ci]['SocialSupport']-0.17,df.iloc[ci]['LifeLadder']+0.05,  country)
    else:
        ax.text(  df.iloc[ci]['SocialSupport']+0.005,df.iloc[ci]['LifeLadder']+0.05,  country) 
ax.set_xticks(np.arange(0,1.2,step=0.2))
ax.set_yticks(np.arange(11,step=2))
ax.set_ylabel('Life Satisfaction (y)')
ax.set_xlabel('Social support (s)')

#Plot model.
ax.plot(x, y, linestyle='-', color='black')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylim(2,8)
ax.set_xlim(0.4,1.05) 
plt.show()

## c)
The code below fits a linear regression model to predict life ladder (happiness) as a linear function of six variables. Use AIC as a manual tool to investigate what the best model is combining these factors.

In [ ]:
factors =['LogGDP', 'SocialSupport', 'LifeExp',  'Freedom', 'Generosity', 'Corruption']

#Combine factors to string 'factor1 + factor2 + factor3 + ...'
regfactors = ' + '.join(factors)

#Regression model over all the data, so use happy NOT df
model_fit=smf.ols(formula='LifeLadder ~ ' + regfactors, data=df).fit()
print(model_fit.summary())     
b=model_fit.params

## d)
Write an automated code to find the model with the smallest AIC= (loglikelihood - num factors)

The best model for five factors is:
```
Loglikelihood - 5 = -110.5140
```

With parameters:

|Parameter|Value|
|---|---|
|Intercept |        50.179034|
|LifeExp |          -2.267705|
|Freedom |           1.761611|
|LifeExp2 |          0.034706|
|SocialSupport3 |    2.351361|
|LifeExp3 |         -0.000170|

This can very likely be improved on.

# 9.2 Analysing goals in football

In this exercise, we will consider the data set `data/shots.csv`. This is a collection of all shots and goals in the English premier league for one season. See: https://figshare.com/articles/dataset/Events/7770599

## Data
'Goal' 1 if a goal, 0 if not a goal
'X' x-location along long side of pitch in co-ordinates (0-100)
'Y' y-location along short side of pitch (where goal is) in co-ordinates (0-100)
'Distance' is distance (in metres) from middle of goal.
'Angle' is of a triangle created fom the shot point to the goal mouth (as descibed in lectures).

In [ ]:
#Load data for all shots
#shots_model=pd.read_csv('data/shots.csv')
shots_model = pd.read_csv('https://uu-sml.github.io/course-sml-public/data/shots.csv')
shots_model.head()

Function for plotting goal mouth 

In [ ]:
def createGoalMouth():
    #Adopted from FC Python
    #Create figure
    fig=plt.figure()
    ax=fig.add_subplot(1,1,1)

    linecolor='black'

    #Pitch Outline & Centre Line
    plt.plot([0,65],[0,0], color=linecolor)
    plt.plot([65,65],[50,0], color=linecolor)
    plt.plot([0,0],[50,0], color=linecolor)
    
    #Left Penalty Area
    plt.plot([12.5,52.5],[16.5,16.5],color=linecolor)
    plt.plot([52.5,52.5],[16.5,0],color=linecolor)
    plt.plot([12.5,12.5],[0,16.5],color=linecolor)
    
    #Left 6-yard Box
    plt.plot([41.5,41.5],[5.5,0],color=linecolor)
    plt.plot([23.5,41.5],[5.5,5.5],color=linecolor)
    plt.plot([23.5,23.5],[0,5.5],color=linecolor)
    
    #Goal
    plt.plot([41.5-5.34,41.5-5.34],[-2,0],color=linecolor)
    plt.plot([23.5+5.34,41.5-5.34],[-2,-2],color=linecolor)
    plt.plot([23.5+5.34,23.5+5.34],[0,-2],color=linecolor)
    
    #Prepare Circles
    leftPenSpot = plt.Circle((65/2,11),0.8,color=linecolor)
    
    #Draw Circles
    ax.add_patch(leftPenSpot)
    
    #Prepare Arcs
    leftArc = Arc((32.5,11),height=18.3,width=18.3,angle=0,theta1=38,theta2=142,color=linecolor)
    
    #Draw Arcs
    ax.add_patch(leftArc)
    
    #Tidy Axes
    plt.axis('off')
    
    return fig,ax

## a)
The code plot the frequency of the data. 

In [ ]:
#Two dimensional histogram
H_Shot=np.histogram2d(shots_model['X'], shots_model['Y'],bins=50,range=[[0, 100],[0, 100]])
goals_only=shots_model[shots_model['Goal']==1]
H_Goal=np.histogram2d(goals_only['X'], goals_only['Y'],bins=50,range=[[0, 100],[0, 100]])

#Plot the number of shots from different points
(fig,ax) = createGoalMouth()
pos=ax.imshow(H_Shot[0], extent=[-1,66,104,-1], aspect='auto',cmap=plt.cm.Reds)
fig.colorbar(pos, ax=ax)
ax.set_title('Number of shots')
plt.xlim((-1,66))
plt.ylim((-3,35))
plt.tight_layout()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
#Plot the number of GOALS from different points
(fig,ax) = createGoalMouth()
pos=ax.imshow(H_Goal[0], extent=[-1,66,104,-1], aspect='auto',cmap=plt.cm.Reds)
fig.colorbar(pos, ax=ax)
ax.set_title('Number of goals')
plt.xlim((-1,66))
plt.ylim((-3,35))
plt.tight_layout()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
#Plot the probability of scoring from different points
(fig,ax) = createGoalMouth()
pos=ax.imshow(H_Goal[0]/H_Shot[0], extent=[-1,66,104,-1], aspect='auto',cmap=plt.cm.Reds,vmin=0, vmax=0.5)
fig.colorbar(pos, ax=ax)
ax.set_title('Proportion of shots resulting in a goal')
plt.xlim((-1,66))
plt.ylim((-3,35))
plt.tight_layout()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

## b)
The code below plots how shot angle determine probability of scoring. It fits a logistic regression model and compares it to data. Make a similar plot for distance to goal. See what happens when you add distance squared.

In [ ]:
#Look at frequencies of goals as function of angle.
shotcount_dist=np.histogram(shots_model['Angle']*180/np.pi,bins=40,range=[0, 150])
goalcount_dist=np.histogram(goals_only['Angle']*180/np.pi,bins=40,range=[0, 150])
prob_goal=np.divide(goalcount_dist[0],shotcount_dist[0])
angle=shotcount_dist[1]
theangle= (angle[:-1] + angle[1:])/2

#Make single variable model of angle
#Using logistic regression we find the optimal values of b
test_model = smf.glm(formula="Goal ~ Angle" , data=shots_model, 
                     family=sm.families.Binomial()).fit()
print(test_model.summary())        
b=test_model.params
xGprob=1/(1+np.exp(-b[0]-b[1]*theangle*np.pi/180)) 

fig,ax=plt.subplots(num=1)
ax.plot(theangle, prob_goal, linestyle='none', marker= '.', markerSize= 12, color='black')
ax.plot(theangle, xGprob, linestyle='solid', color='black')
ax.set_ylabel('Probability chance scored')
ax.set_xlabel("Shot angle (degrees)")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()

In [ ]:
#Show empirically how distance from goal predicts probability of scoring
#Make single variable model of distance


In [ ]:
#Adding distance squared


## c)
By setting `model_variables` in the code below you can test different features. Investigate manually which parameters work best. 

In [ ]:
#Adding even more variables to the model.
squaredX = shots_model['X']**2
shots_model = shots_model.assign(X2=squaredX)
squaredC = shots_model['C']**2
shots_model = shots_model.assign(C2=squaredC)
AX = shots_model['Angle']*shots_model['X']
shots_model = shots_model.assign(AX=AX)

# A general model for fitting goal probability
# List the model variables you want here
model_variables = ['Distance']

#Combine factors to string 'factor1 + factor2 + factor3 + ...'
model = ' + '.join(model_variables)

#Fit the model
test_model = smf.glm(formula="Goal ~ " + model, data=shots_model,
                     family=sm.families.Binomial()).fit()
print(test_model.summary())
b=test_model.params

In [ ]:
#Return xG value for more general model
def calculate_xG(sh):    
   bsum=b[0]
   for i,v in enumerate(model_variables):
       bsum=bsum+b[i+1]*sh[v]
   xG = 1/(1+np.exp(-bsum)) 
   return xG

#Add an xG to my dataframe
xG=shots_model.apply(calculate_xG, axis=1) 
shots_model = shots_model.assign(xG=xG)

#Create a 2D map of xG
pgoal_2d=np.zeros((65,65))
for x in range(65):
    for y in range(65):
        sh=dict()
        a = np.arctan(7.32 *x /(x**2 + abs(y-65/2)**2 - (7.32/2)**2))
        if a<0:
            a = np.pi + a
        sh['Angle'] = a
        sh['Distance'] = np.sqrt(x**2 + abs(y-65/2)**2)
        sh['D2'] = x**2 + abs(y-65/2)**2
        sh['X'] = x
        sh['AX'] = x*a
        sh['X2'] = x**2
        sh['C'] = abs(y-65/2)
        sh['C2'] = (y-65/2)**2
        
        pgoal_2d[x,y] =  calculate_xG(sh)

(fig,ax) = createGoalMouth()
pos=ax.imshow(pgoal_2d, extent=[-1,65,65,-1], aspect='auto',cmap=plt.cm.Reds,vmin=0, vmax=0.3)
fig.colorbar(pos, ax=ax)
ax.set_title('Probability of goal')
plt.xlim((0,66))
plt.ylim((-3,35))
plt.gca().set_aspect('equal', adjustable='box')
plt.show()